# Rete Neurale per Predire Acquisti
Questo progetto crea e addestra una rete MLP per predire se un utente acquisterà un prodotto in base a dati demografici e comportamentali.

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)
n = 500
eta = np.random.randint(18, 66, size=n)
salario = np.random.normal(loc=2000, scale=600, size=n).astype(int)
genere = np.random.choice(['M', 'F'], size=n)
stato_civile = np.random.choice(['Single', 'Sposato'], size=n)
visite_sito = np.random.poisson(lam=3, size=n)
acquisto = [
    1 if (v > 2 and s > 1800 and e < 50) else 0
    for v, s, e in zip(visite_sito, salario, eta)
]
df = pd.DataFrame({
    'Eta': eta,
    'Salario': salario,
    'Genere': genere,
    'Stato_civile': stato_civile,
    'Visite_sito': visite_sito,
    'Acquisto': acquisto
})
df.to_csv('clienti_acquisti.csv', index=False)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('clienti_acquisti.csv')
X = df.drop('Acquisto', axis=1)
y = df['Acquisto']
numeric_features = ['Eta', 'Salario', 'Visite_sito']
categorical_features = ['Genere', 'Stato_civile']
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(), categorical_features)
])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Andamento della Loss')
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Andamento dell’Accuracy')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
import ipywidgets as widgets
from IPython.display import display

eta_widget = widgets.IntSlider(value=30, min=18, max=65, description='Età:')
salario_widget = widgets.IntSlider(value=2000, min=500, max=5000, step=100, description='Salario:')
genere_widget = widgets.Dropdown(options=['M', 'F'], description='Genere:')
stato_civile_widget = widgets.Dropdown(options=['Single', 'Sposato'], description='Stato Civile:')
visite_widget = widgets.IntSlider(value=3, min=0, max=10, description='Visite Sito:')

def predizione_interattiva(change=None):
    nuovo_df = pd.DataFrame([{
        'Eta': eta_widget.value,
        'Salario': salario_widget.value,
        'Genere': genere_widget.value,
        'Stato_civile': stato_civile_widget.value,
        'Visite_sito': visite_widget.value
    }])
    dati_proc = preprocessor.transform(nuovo_df)
    prob = model.predict(dati_proc)[0][0]
    classe = int(prob > 0.5)
    print(f"\n📊 Dati inseriti: {nuovo_df.to_dict(orient='records')[0]}")
    print(f"📈 Probabilità di acquisto: {prob:.2f}")
    print(f"🛍️ Predizione: {'ACQUISTERÀ' if classe else 'NON acquisterà'} il prodotto.")

bottone = widgets.Button(description="Predici")
bottone.on_click(predizione_interattiva)
display(eta_widget, salario_widget, genere_widget, stato_civile_widget, visite_widget, bottone)